In [1]:
!pip3 install ultralytics
!pip3 install lapx>=0.5.2

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 906 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 56.5 MB 4.7 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
zsh:1: 0.5.2 not found


In [1]:
from ultralytics import YOLO
import cv2
from paddleocr import PaddleOCR
import numpy as np
from scipy.ndimage import rotate
from lime_enhancement import LIME
from weighted_levenshtein import lev
import numpy as np

/Users/shravanprasanth/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [70]:
ocr = PaddleOCR(use_angle_cls=False, lang='en')
lime = LIME()

[2025/01/13 22:34:17] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/shravanprasanth/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/shravanprasanth/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batc

In [4]:
model = YOLO('/Users/shravanprasanth/Coding/Robotics/Scarlett-AI/src/models/y11v9.pt')

In [8]:
!pip3 install lap

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.5 MB 3.3 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
original_red_team_numbers = ["75", "2722", "1391"]
original_blue_team_numbers = ["56", "5401", "8513"]

In [6]:
substitute_costs = np.ones((128, 128), dtype=np.float64)

# Define similar characters
similarities = [
    ("S", "5"),
    ("A", "4"),
    ("D", "0"),
    ("I", "1"),
    ("O", "0"),
    ("R", "6"),
    ("n", "0"),
    ("7", "1"),
    ("T", "7"),
    ("B", "5"),
    ("S", "2"),
    ("Q", "0"),
]

for similarity1, similarity2 in similarities:
    substitute_costs[ord(similarity1), ord(similarity2)] = 0.5
    substitute_costs[ord(similarity2), ord(similarity1)] = 0.5


In [34]:
!pip3 uninstall opencv-python -y
!pip3 uninstall opencv-contrib-python -y
!pip3 install opencv-contrib-python

Found existing installation: opencv-contrib-python 4.10.0.84
Uninstalling opencv-contrib-python-4.10.0.84:
  Successfully uninstalled opencv-contrib-python-4.10.0.84
Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-macosx_12_0_x86_64.whl (66.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddleocr 2.9.1 requires opencv-python, which is not installed.
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:

upscale_model = cv2.dnn_superres.DnnSuperResImpl_create()
upscale_model.readModel('/Users/shravanprasanth/Coding/Robotics/Scarlett-AI/src/models/ESPCN_x4.pb')
upscale_model.setModel('espcn', 4)

In [78]:
import os
import random

In [102]:
video_source = '/Users/shravanprasanth/Coding/Robotics/Scarlett-AI/src/videos/lehigh58-auto.mp4'
output_path = '/Users/shravanprasanth/Coding/Robotics/Scarlett-AI/src/videos/output/lehigh58-auto-output.mp4'

# Capture the video source to get frame properties
video_capture = cv2.VideoCapture(video_source)
fps = video_capture.get(cv2.CAP_PROP_FPS)
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up the video writer to save the output video
video_writer = cv2.VideoWriter(
    output_path, 
    cv2.VideoWriter_fourcc(*'mp4v'), 
    fps, 
    (frame_width, frame_height)
)

# Run the tracking process
results = model.track(
    source=video_source,
    save=False,  # Avoid auto-saving
    tracker='/Users/shravanprasanth/Coding/Robotics/Scarlett-AI/src/config/bytetrack.yaml',
    persist=False
)
first_frame_with_six_robots = False
robot_team_numbers_by_id = {}

# Process each frame and save it to the output video
for result in results:
    frame = result.orig_img  # Original frame from the video
    red_count = 0
    blue_count = 0
    for bbox in result.boxes:
        class_label = result.names[int(bbox.cls[0])]  # Get class label
        if class_label == 'Blue':
            blue_count += 1
        elif class_label == 'Red':
            red_count += 1
    
    if red_count == 3 and blue_count == 3 and first_frame_with_six_robots == False:
        first_frame_with_six_robots = True
        ## FIRST TIME WITH 3 ROBOTS SO DO OCR

        first_red_ocr_results = []
        first_blue_ocr_results = []


        for bbox in result.boxes:
            class_label = (result.names[int(bbox.cls[0])])
            x1, y1, x2, y2 = map(int, bbox.xyxy[0])
            robot_id = str([int(bbox.id[0])])  # Get class label
            start_row = (y1 + y2) // 2
            bottom_half = frame[start_row:y2, x1:x2]

            # Determine color based on the class label
            if class_label == 'Blue':
                color = (255, 0, 0)  # Blue color in BGR format
                blue_count += 1
                available_team_numbers = original_blue_team_numbers.copy()
            elif class_label == 'Red':
                color = (0, 0, 255)  # Red color in BGR format
                red_count += 1
                available_team_numbers = original_red_team_numbers.copy()
            else:
                color = (0, 255, 0)  # Default color (green) for other classes


            # OCR: Recognize text in the bottom half of the box
            height, width, _ = bottom_half.shape
            upscaled_bottom_half = upscale_model.upsample(bottom_half)
            gray = cv2.cvtColor(upscaled_bottom_half, cv2.COLOR_BGR2GRAY)

            # Apply LIME enhancement
            lime.load(gray)
            enhanced_image = lime.enhance()

            
            ocr_result = ocr.ocr(np.array(enhanced_image))
            try:
                oscrr = ocr_result[0][0][1][0]
            except:
                enhanced_image = rotate(enhanced_image, -20)
                ocr_result = ocr.ocr(np.array(enhanced_image))
                try:
                    oscrr = ocr_result[0][0][1][0]
                except:
                    enhanced_image = rotate(enhanced_image, 40)
                    ocr_result = ocr.ocr(np.array(enhanced_image))
                    try:
                        oscrr = ocr_result[0][0][1][0]
                    except:
                        oscrr = ""
            detected_text = oscrr.replace(" ", "") if oscrr else ""

            distances = {num: lev(detected_text, num, substitute_costs=substitute_costs) for num in available_team_numbers}
            sorted_distances = sorted(distances.items(), key=lambda item: item[1])
            
            closest_robot_match = sorted_distances[0][0]
            closest_robot_match_distance = sorted_distances[0][1]


            if class_label == "Blue":
                first_blue_ocr_results.append({
                    "x1": x1,
                    "x2": x2,
                    "y1": y1,
                    "y2": y2,
                    "color": (255,0,0),
                    "sorted_distances": sorted_distances,
                    "robot_id": robot_id
                })
            else:
                first_red_ocr_results.append({
                    "x1": x1,
                    "x2": x2,
                    "y1": y1,
                    "y2": y2,
                    "color": (0,0,255),
                    "sorted_distances": sorted_distances,
                    "robot_id": robot_id
                })

        for team in first_red_ocr_results:
            available_team_numbers = original_red_team_numbers.copy()
            i = 0
            while team['sorted_distances'][i][0] not in available_team_numbers and i < 3:
                i += 1
            
            if i < 3:
                robot_team_numbers_by_id[team['robot_id']] = team['sorted_distances'][i][0]
                
                cv2.rectangle(frame, (team['x1'], team['y1']), (team['x2'], team['y2']), team['color'], 2)
                cv2.putText(frame, f"{team['sorted_distances'][i][0]}", (team['x1'], team['y1'] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, team['color'], 2)
        
                available_team_numbers.remove(team['sorted_distances'][i][0])
        
        for team in first_blue_ocr_results:
            available_team_numbers = original_blue_team_numbers.copy()
            i = 0
            while team['sorted_distances'][i][0] not in available_team_numbers and i < 3:
                i += 1
            
            if i < 3:
                robot_team_numbers_by_id[team['robot_id']] = team['sorted_distances'][i][0]

                cv2.rectangle(frame, (team['x1'], team['y1']), (team['x2'], team['y2']), team['color'], 2)
                cv2.putText(frame, f"{team['sorted_distances'][i][0]}", (team['x1'], team['y1'] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, team['color'], 2)
        
                available_team_numbers.remove(team['sorted_distances'][i][0])
        
    
    
    
    elif first_frame_with_six_robots == True:
        for bbox in result.boxes:
            # Extract bounding box coordinates and class label
            x1, y1, x2, y2 = map(int, bbox.xyxy[0])  # Bounding box coordinates
            class_label = result.names[int(bbox.cls[0])]  # Get class label

            robot_id = str([int(bbox.id[0])])  # Get class label

            if robot_team_numbers_by_id.get(robot_id, "") == "":
                continue
            else:
                team_number = robot_team_numbers_by_id.get(robot_id, "")
                if class_label == 'Blue':
                    color = (255, 0, 0)
                else:
                    color = (0, 0, 255)

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{team_number}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        

    # Write the modified frame to the output video
    video_writer.write(frame)

# Release resources
video_capture.release()
video_writer.release()
cv2.destroyAllWindows()

print(f"Processed video saved to {output_path}")



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1413) /Users/shravanprasanth/Coding/Robotics/Scarlett-AI/src/videos/lehigh58-auto.mp4: 384x640 2 Blues, 2 Reds, 81.5ms
video 1/1 (frame 2/1413) /Users/shravanprasanth/Coding/Robotics/Scarlett-AI/src/videos/lehigh58-auto.mp4: 384x640 3 Blues, 3 Reds, 50.9ms
video 1/1 (frame 3/1413) /Users/shravanprasanth/Coding/Robotics/Scarlett-AI/src/videos/lehigh58-auto.mp4: 384x640 3 Blues, 3 Reds, 48.6ms
video 1/1 (frame 4/1413) /Users/shravanpra